# Imports

In [138]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist


# Input

In [139]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [140]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [141]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [142]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [143]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.16800388 0.16810163 0.16800388 ... 0.15640596 0.22703599 0.15640596]
[0.38020883 0.38043003 0.38020883 ... 0.35396161 0.51380412 0.35396161]
[0.57421208 0.57425462 0.57421208 ... 0.56915696 0.59968152 0.56915696]
(5999,)


### Multinomial Sampling

In [144]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[1 0 0 ... 1 0 0]


In [145]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[78.63775  ,  7.6759295, 83.84298  ],
       [92.00067  ,  2.9401166, 82.98684  ],
       [77.63564  ,  3.5589852, 97.840775 ],
       [76.761116 ,  3.5589852, 97.95591  ],
       [83.54716  ,  2.9401166, 96.00411  ]], dtype=float32)

# KNN extended graph

In [146]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [147]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11845


# Edge Predictor

In [148]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [149]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [150]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [151]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [152]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 13)	1
  (1, 14)	1
  :	:
  (1498, 1310)	1
  (1498, 1335)	1
  (1498, 1336)	1
  (1498, 1337)	1
  (1498, 1338)	1
  (1498, 1339)	1
  (1498, 1345)	1
  (1498, 1346)	1
  (1498, 1372)	1
  (1498, 1499)	1
  (1499, 1303)	1
  (1499, 1304)	1
  (1499, 1305)	1
  (1499, 1306)	1
  (1499, 1331)	1
  (1499, 1332)	1
  (1499, 1333)	1
  (1499, 1334)	1
  (1499, 1335)	1
  (1499, 1336)	1
  (1499, 1337)	1
  (1499, 1360)	1
  (1499, 1372)	1
  (1499, 1381)	1
  (1499, 1498)	1
[[0.07238728 0.07201284 0.07201284 ... 0.07556467 0.07394938 0.07384532]
 [1.0371147  0.07089981 0.07089981 ... 0.0677008  0.06915219 0.06924589]
 [1.03698191 0.52571794 0.07086785 ... 0.06747974 0.06901598 0.06911522]
 ...
 [1.14454819 1.1323522  1.13630509 ... 0.03966688 0.05308191 0.

In [153]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [154]:
print(A_s)  # symmétrique
print(A_s.shape)

[[1.59716670e-30 1.47099904e-30 1.47280398e-30 ... 6.12050254e-28
  9.42825355e-29 8.02796310e-29]
 [1.47099904e-30 1.36831041e-30 1.37593572e-30 ... 4.38224777e-28
  7.43145911e-29 6.39106468e-29]
 [1.47280398e-30 1.37593572e-30 1.38357060e-30 ... 4.33907422e-28
  7.38953104e-29 6.35763188e-29]
 ...
 [6.12050254e-28 4.38224777e-28 4.33907422e-28 ... 2.52225179e-24
  2.30080723e-25 1.83376260e-25]
 [9.42825355e-29 7.43145911e-29 7.38953104e-29 ... 2.30080723e-25
  2.27823093e-26 1.83487502e-26]
 [8.02796310e-29 6.39106468e-29 6.35763188e-29 ... 1.83376260e-25
  1.83487502e-26 1.48009148e-26]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [155]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

IGRAPH U--- 1500 11845 -- 
+ attr: _nx_name (v)


In [156]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[ 821  853  854]
 [1100 1101 1448]
 [1100 1436 1448]]
33322


In [157]:
triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[77.13215    -6.9390492   1.1670189 ]
  [77.9193     -7.328478    1.171451  ]
  [77.309      -7.328478    0.62193453]]

 [[75.37597    -6.0434117  83.57376   ]
  [75.79908    -5.5423326  82.99476   ]
  [77.934105   -6.9390492  82.894005  ]]

 [[75.37597    -6.0434117  83.57376   ]
  [78.17806    -7.328478   83.509     ]
  [77.934105   -6.9390492  82.894005  ]]]
33322


In [158]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(33322,)
[1.87080936e-26 2.64830410e-23 1.21750010e-25 ... 1.14313975e-17
 2.22009081e-26 3.60545294e-23]


#### Calculate barycenter

In [159]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(33322, 3)
[[77.45348    -7.198668    0.98680145]
 [76.36972    -6.174931   83.154175  ]
 [77.16271    -6.770313   83.32559   ]]


#### KNN Tri

In [160]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [161]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(33322, 19, 3)


#### calculate e norm matrix

In [162]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(33322, 3)
[[0.87822294 0.69285446 0.82123816]
 [0.87484103 2.7943325  2.553292  ]]


#### Calculate r

In [163]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(33322,)

In [164]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(33322, 19)
(33322, 19)


In [165]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(33322, 19, 5)


#### Calculate f

In [166]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.000000000000102
(33322,)
[2.99538122e-05 2.99538122e-05 2.99538122e-05 ... 2.99538122e-05
 2.99538122e-05 2.99538122e-05]


C:\Users\ahdel\AppData\Local\Temp\ipykernel_1692\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [167]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[ 39.          1.4916434   3.6726582]
 [ 51.         -6.3966537   1.0434115]
 [ 39.          6.1633806 -11.281142 ]]


In [168]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 47
Number of edges: 232


In [169]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [170]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [171]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[78.63775    7.6759295 83.84298  ]
 [78.17806    7.328478  83.509    ]
 [78.294556   7.328478  83.33951  ]
 ...
 [85.57074    8.114116  -2.3030212]
 [83.32611   -8.672658  87.249275 ]
 [84.99718   -8.535009  85.65103  ]]
dim sample points :  1500


Distance Matrix

In [172]:
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean').T

distances_matrix = matrix_distance_euclidienne_barycentres(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 6489.50906941  4588.14514589  6479.03167399 ...  9276.96955552
   9969.97131747  9162.66534823]
 [ 6427.36986775  4534.72789147  6417.33178533 ...  9180.04514058
   9862.3642862   9066.09078767]
 [ 6401.12624542  4512.85979762  6391.09403424 ...  9155.74429489
   9841.08101708  9042.01168114]
 ...
 [  225.53750857   546.74766944   214.82627971 ...  1495.51757115
   2467.72031284  1494.02255893]
 [ 7096.8007143   5113.09109388  7108.32065854 ...  9924.40210777
  10721.37159831  9801.54493816]
 [ 6860.96035382  4918.47649618  6872.3813001  ...  9736.57181896
  10575.77227475  9615.84002377]]


In [173]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [174]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [175]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00018439 0.00018441 0.00018439 ... 0.00018277 0.00019257 0.00018277]


In [176]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 5.20522107966978


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [177]:
# Creation igraph
igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH U--- 5999 17991 -- 
+ attr: _nx_name (v), index_triangle (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [178]:
# Get finals selected triangles & calculate barycenters
b_hat = np.mean(selected_triangles, axis=1)
print(b_hat.shape)
print("B_HAT : ", b_hat)

(500, 3)
B_HAT :  [[43.         0.4194568 -2.1883576]
 [59.        -7.537207  -3.5616977]
 [59.        -7.4618874 -2.90124  ]
 ...
 [55.        -2.4471858 92.70333  ]
 [55.        -1.7850233 89.24637  ]
 [47.        -2.6524875 90.53451  ]]


test matrice des normes euclidiennes entre b et b_hat

In [179]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(500, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[  77.42136611  179.44193946  224.22017013 ... 1567.14723663
  1131.34326099 1783.16030385]
 [ 524.46016864  142.95427322   17.53139386 ...  904.58124131
   443.52243323 1136.59318537]
 [ 524.19521793  132.46890692   17.12033759 ...  871.5856057
   427.4832888  1097.88431994]
 ...
 [9794.62023078 7928.26208343 9158.55052064 ... 5545.88345935
  7562.48811441 4947.59825834]
 [9117.37663595 7320.87178261 8518.64342057 ... 5060.52686599
  6987.20942276 4491.67886197]
 [9252.6606951  7612.51542241 8810.52597776 ... 5625.12206181
  7585.31018369 5045.24392187]]


In [180]:
minimums_b = np.min(distances, axis=1)
print(minimums_b.shape)
print(minimums_b[:5])

(500,)
[19.18900994  1.26901563  1.08858186 18.71240331 22.03458036]


In [181]:
d_f_S_Ss = np.sum(selected_triangles_indexes * minimums_b)
print(d_f_S_Ss)

211042255.2828564


matrice sqeuclidean b

In [182]:
def normes_euclidiennes_carrees_b(bar_b):
    # Calculer la matrice des normes euclidiennes au carré
    normes_carrees = np.sum(bar_b**2, axis=1)

    return normes_carrees

matrix_b = normes_euclidiennes_carrees_b(b)
print(matrix_b.shape)
print(matrix_b)

(11994,)
[1950.2314 3040.0283 3105.3853 ... 6075.3086 5692.7305 6270.884 ]


In [183]:
def matrix_distances_after_knn(b_hat_n, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = knn_and_extended_graph(b_hat_n, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

Distances After k-NN:
 [[ 0.          0.39912523  0.52494784 ... 98.20813801 98.00814387
  95.67155228]
 [ 0.39912523  0.          0.16979431 ... 98.22689689 98.02729009
  95.6719752 ]
 [ 0.52494784  0.16979431  0.         ... 98.36147026 98.16196212
  95.8016635 ]
 ...
 [98.20813801 98.22689689 98.36147026 ...  0.          0.22200676
   5.1346762 ]
 [98.00814387 98.02729009 98.16196212 ...  0.22200676  0.
   5.11729035]
 [95.67155228 95.6719752  95.8016635  ...  5.1346762   5.11729035
   0.        ]]


In [184]:
def probabilistic_surface_distance_extended(b_i, b_hat_i, p_b_hat_i):
    """
    Calculate Probabilistic Surfaces Distance 

    Parameters:
    - b: array-like, shape (N, 3)
        Barycenters of the ground truth surface.
    - b_hat: array-like, shape (M, 3)
        Barycenters of the generated triangles.
    - p_b_hat: array-like, shape (M,)
        Probabilities corresponding to the generated triangles.

    Returns:
    - d_S_Ss: float
        Probabilistic Surfaces Distance.
    """
    # Debugging print statements
    print("b shape:", b_i.shape)
    print("b_hat shape:", b_hat_i.shape)
    print("p_b_hat shape:", p_b_hat_i.shape)

    p_b_hat_i = normalized_inclusion_score.reshape((-1, 1))
    dist_matrixs_1= distances_after - matrix_b
    #dist_matrixs_2= matrix_b - distances_after

    d_S_Ss = np.sum(p_b_hat_i * np.min(np.sum(dist_matrixs_1)**2))

    

    return d_S_Ss

result_distance = probabilistic_surface_distance_extended(b, b_hat, normalized_inclusion_score)
print("Probabilistic Surfaces Distance:", result_distance)

b shape: (11994, 3)
b_hat shape: (500, 3)
p_b_hat shape: (5999,)


ValueError: operands could not be broadcast together with shapes (500,500) (11994,) 

## Triangle Collision Loss

In [ ]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

